# Lean 3 - Propositions et Preuves

## Introduction

Ce notebook explore la caracteristique la plus puissante de Lean : sa capacite a representer des **propositions mathematiques** comme des types et des **preuves** comme des termes. Cette correspondance, connue sous le nom d'**isomorphisme de Curry-Howard**, est le fondement de la verification formelle.

### Objectifs pedagogiques

1. Comprendre le type special `Prop` pour les propositions
2. Maitriser l'isomorphisme de Curry-Howard (propositions = types, preuves = termes)
3. Manipuler les connecteurs logiques : `->`, `And`, `Or`, `Not`, `Iff`
4. Construire des preuves par manipulation de termes
5. Utiliser l'egalite et les preuves calculatoires avec `calc`
6. Distinguer logique constructive et logique classique

### Prerequis

- Avoir complete les notebooks **Lean-1-Setup** et **Lean-2-Dependent-Types**
- Notions de base en logique propositionnelle

### Duree estimee : 45-50 minutes

---

## L'isomorphisme de Curry-Howard

### Le principe fondamental

Haskell Curry et William Howard ont independamment decouvert une correspondance profonde entre logique et programmation :

| Logique | Programmation (types) |
|---------|----------------------|
| Proposition | Type |
| Preuve | Terme (valeur) du type |
| Implication `P -> Q` | Type fonction `P -> Q` |
| Conjonction `P et Q` | Type produit `P x Q` |
| Disjonction `P ou Q` | Type somme `P + Q` |
| Vrai | Type `Unit` (un seul habitant) |
| Faux | Type `Empty` (aucun habitant) |

**Consequence** : Prouver une proposition revient a construire un terme du type correspondant. Si le type est habite (il existe un terme), la proposition est vraie.

## 1. Le Type `Prop`

### 1.1 Introduction a `Prop`

En Lean, `Prop` est un univers special qui contient les **propositions** - des enonces qui peuvent etre vrais ou faux. Contrairement a `Type`, les propositions ont une semantique logique : ce qui compte n'est pas la forme de la preuve, mais son **existence**.

In [ ]:
-- Prop est l'univers des propositions
#check Prop          -- Prop : Type

-- Quelques propositions de base
#check True          -- True : Prop (proposition toujours vraie)
#check False         -- False : Prop (proposition toujours fausse)

-- Declarer des variables propositionnelles
variable (p q r : Prop)

#check p             -- p : Prop
#check p -> q        -- p -> q : Prop (implication)

### 1.2 Propositions vs Types

| Aspect | `Prop` | `Type` |
|--------|--------|--------|
| Semantique | Verite/Faussete | Donnees/Calcul |
| Proof irrelevance | Oui (toutes les preuves sont egales) | Non |
| Usage | Raisonnement logique | Programmation |
| Extraction | Effacee a la compilation | Conservee |

In [ ]:
-- True a une preuve triviale
#check True.intro    -- True.intro : True

-- False n'a pas de preuve (type vide)
-- False.elim permet d'en deduire n'importe quoi (ex falso quodlibet)
#check @False.elim   -- False.elim : {C : Sort u} -> False -> C

-- Exemple de theorem trivial
theorem trivial_true : True := True.intro

## 2. L'Implication : Preuves comme Fonctions

### 2.1 L'implication `P -> Q`

En Lean, l'implication `P -> Q` est representee par le type fonction. Une **preuve** de `P -> Q` est une **fonction** qui transforme toute preuve de `P` en preuve de `Q`.

C'est l'essence de Curry-Howard : **prouver une implication = programmer une fonction**.

In [ ]:
-- Variables propositionnelles
variable (p q : Prop)

-- Theoreme : p implique p (reflexivite)
-- La preuve est la fonction identite!
theorem impl_refl : p -> p :=
  fun hp : p => hp

#check impl_refl     -- impl_refl : p -> p

-- Theoreme : p implique (q implique p)
-- C'est la fonction constante!
theorem impl_intro : p -> q -> p :=
  fun hp : p =>
    fun hq : q => hp

#check impl_intro    -- impl_intro : p -> q -> p

### 2.2 Transitivite de l'implication

Si `P -> Q` et `Q -> R`, alors `P -> R`. C'est la **composition de fonctions**!

In [ ]:
variable (p q r : Prop)

-- Transitivite = composition de fonctions
theorem impl_trans (hpq : p -> q) (hqr : q -> r) : p -> r :=
  fun hp : p =>
    hqr (hpq hp)     -- Appliquer hpq a hp, puis hqr au resultat

-- Version avec application de fonction explicite
theorem impl_trans' (hpq : p -> q) (hqr : q -> r) : p -> r :=
  fun hp : p =>
    let hq : q := hpq hp    -- Obtenir preuve de q
    let hr : r := hqr hq    -- Obtenir preuve de r
    hr

#check @impl_trans   -- impl_trans : (p -> q) -> (q -> r) -> p -> r

## 3. La Conjonction : `And` (et logique)

### 3.1 Definition de `And`

La conjonction `P And Q` (ou `P /\ Q`) est vraie si et seulement si `P` et `Q` sont toutes deux vraies. C'est un **type produit** : une preuve de `P /\ Q` contient a la fois une preuve de `P` et une preuve de `Q`.

In [ ]:
variable (p q : Prop)

-- And est un type produit pour les propositions
#check And           -- And : Prop -> Prop -> Prop
#check p /\ q        -- p /\ q : Prop (notation pour And p q)

-- Constructeur : And.intro combine deux preuves
#check @And.intro    -- And.intro : p -> q -> p /\ q

-- Destructeurs : And.left et And.right extraient les composantes
#check @And.left     -- And.left : p /\ q -> p
#check @And.right    -- And.right : p /\ q -> q

### 3.2 Preuves avec la conjonction

Pour prouver une conjonction `P /\ Q`, on doit fournir des preuves des deux parties separement. Pour l'utiliser, on peut extraire chaque partie avec `.left` et `.right`.

**Pattern d'introduction** : `And.intro hp hq` ou `⟨hp, hq⟩`
**Pattern d'elimination** : `h.left` et `h.right`

In [ ]:
variable (p q : Prop)

-- Introduction de la conjonction
theorem and_intro_example (hp : p) (hq : q) : p /\ q :=
  And.intro hp hq

-- Syntaxe avec chevrons (angle brackets)
theorem and_intro_example' (hp : p) (hq : q) : p /\ q :=
  ⟨hp, hq⟩           -- Equivalent a And.intro

-- Elimination : extraire les composantes
theorem and_left_example (hpq : p /\ q) : p :=
  hpq.left           -- Equivalent a And.left hpq

theorem and_right_example (hpq : p /\ q) : q :=
  hpq.right

-- Commutativite de And
theorem and_comm : p /\ q -> q /\ p :=
  fun hpq => ⟨hpq.right, hpq.left⟩

### 3.3 Proprietes de la conjonction

La conjonction possede des proprietes algebriques importantes qui se prouvent en manipulant les composantes :

- **Commutativite** : `p /\ q <-> q /\ p`
- **Associativite** : `(p /\ q) /\ r <-> p /\ (q /\ r)`
- **Idempotence** : `p /\ p <-> p`
- **Absorption** : `p /\ True <-> p`

In [ ]:
variable (p q r : Prop)

-- Associativite
theorem and_assoc : (p /\ q) /\ r <-> p /\ (q /\ r) :=
  ⟨fun h => ⟨h.left.left, ⟨h.left.right, h.right⟩⟩,
   fun h => ⟨⟨h.left, h.right.left⟩, h.right.right⟩⟩

-- Distributivite sur l'implication
theorem and_impl_distrib : (p /\ q -> r) <-> (p -> q -> r) :=
  ⟨fun h hp hq => h ⟨hp, hq⟩,
   fun h hpq => h hpq.left hpq.right⟩

## 4. La Disjonction : `Or` (ou logique)

### 4.1 Definition de `Or`

La disjonction `P Or Q` (ou `P \/ Q`) est vraie si au moins l'une des deux propositions est vraie. C'est un **type somme** : une preuve de `P \/ Q` est soit une preuve de `P`, soit une preuve de `Q`.

In [ ]:
variable (p q : Prop)

#check Or            -- Or : Prop -> Prop -> Prop
#check p \/ q        -- p \/ q : Prop

-- Deux constructeurs : Or.inl et Or.inr (left et right)
#check @Or.inl       -- Or.inl : p -> p \/ q
#check @Or.inr       -- Or.inr : q -> p \/ q

-- Elimination : Or.elim (analyse par cas)
#check @Or.elim      -- Or.elim : p \/ q -> (p -> r) -> (q -> r) -> r

### 4.2 Preuves avec la disjonction

Pour prouver une disjonction `P \/ Q`, il suffit de prouver l'une des deux parties. Pour l'utiliser, on doit traiter les deux cas possibles (analyse par cas).

**Pattern d'introduction** : `Or.inl hp` (gauche) ou `Or.inr hq` (droite)
**Pattern d'elimination** : `match h with | Or.inl hp => ... | Or.inr hq => ...`

In [ ]:
variable (p q r : Prop)

-- Introduction a gauche
theorem or_intro_left (hp : p) : p \/ q :=
  Or.inl hp

-- Introduction a droite
theorem or_intro_right (hq : q) : p \/ q :=
  Or.inr hq

-- Elimination : analyser les deux cas
theorem or_elim_example (hpq : p \/ q) (hpr : p -> r) (hqr : q -> r) : r :=
  Or.elim hpq hpr hqr

-- Version avec match (plus lisible)
theorem or_elim_match (hpq : p \/ q) (hpr : p -> r) (hqr : q -> r) : r :=
  match hpq with
  | Or.inl hp => hpr hp
  | Or.inr hq => hqr hq

-- Commutativite de Or
theorem or_comm : p \/ q -> q \/ p :=
  fun hpq => match hpq with
    | Or.inl hp => Or.inr hp
    | Or.inr hq => Or.inl hq

## 5. La Negation : `Not`

### 5.1 Definition de `Not`

La negation `Not P` (ou `\neg P`) est definie comme `P -> False`. Intuitivement : si on suppose `P`, on arrive a une contradiction.

In [ ]:
variable (p : Prop)

#check Not           -- Not : Prop -> Prop
#check ¬p            -- ¬p : Prop (equivalent a p -> False)

-- Definition de Not
#print Not           -- def Not (a : Prop) : Prop := a -> False

-- Prouver une negation = construire une fonction vers False
theorem not_false : ¬False :=
  fun h : False => h   -- La preuve de False est elle-meme

-- Ex falso quodlibet : de False on deduit tout
theorem false_elim_example (h : False) : p :=
  False.elim h

### 5.2 Contradiction et absurdite

La contradiction est l'outil principal pour prouver des negations. Si on peut deduire `False` d'une hypothese, cette hypothese est fausse.

**Principe cle** : `absurd hp hnp` - de `p` et `¬p` on deduit n'importe quoi (ex falso quodlibet)
**Modus tollens** : de `p -> q` et `¬q`, on deduit `¬p`

In [ ]:
variable (p q : Prop)

-- Modus tollens : (p -> q) -> ¬q -> ¬p
theorem modus_tollens (hpq : p -> q) (hnq : ¬q) : ¬p :=
  fun hp : p =>
    let hq : q := hpq hp   -- On obtient q
    hnq hq                  -- Contradiction avec ¬q

-- Non-contradiction : ¬(p /\ ¬p)
theorem non_contradiction : ¬(p /\ ¬p) :=
  fun h : p /\ ¬p =>
    h.right h.left   -- ¬p applique a p donne False

-- absurd : de p et ¬p on deduit tout
#check @absurd       -- absurd : {a : Prop} -> {b : Sort v} -> a -> ¬a -> b

theorem absurd_example (hp : p) (hnp : ¬p) : q :=
  absurd hp hnp

## 6. L'Equivalence : `Iff`

### 6.1 Definition de `Iff`

L'equivalence `P <-> Q` ("P si et seulement si Q") signifie que `P -> Q` et `Q -> P`. C'est une conjonction de deux implications.

In [ ]:
variable (p q : Prop)

#check Iff           -- Iff : Prop -> Prop -> Prop
#check p <-> q       -- p <-> q : Prop

-- Structure de Iff
#check @Iff.intro    -- Iff.intro : (p -> q) -> (q -> p) -> (p <-> q)
#check @Iff.mp       -- Iff.mp : (p <-> q) -> p -> q (modus ponens)
#check @Iff.mpr      -- Iff.mpr : (p <-> q) -> q -> p (modus ponens reverse)

-- Reflexivite de l'equivalence
theorem iff_refl : p <-> p :=
  ⟨fun hp => hp, fun hp => hp⟩

### 6.2 Preuves d'equivalence

In [ ]:
variable (p q r : Prop)

-- Commutativite de And (version iff)
theorem and_comm_iff : p /\ q <-> q /\ p :=
  ⟨fun h => ⟨h.right, h.left⟩,
   fun h => ⟨h.right, h.left⟩⟩

-- Commutativite de Or (version iff)
theorem or_comm_iff : p \/ q <-> q \/ p :=
  ⟨fun h => h.elim Or.inr Or.inl,
   fun h => h.elim Or.inr Or.inl⟩

-- Transitivite de l'equivalence
theorem iff_trans (hpq : p <-> q) (hqr : q <-> r) : p <-> r :=
  ⟨fun hp => hqr.mp (hpq.mp hp),
   fun hr => hpq.mpr (hqr.mpr hr)⟩

-- Utilisation de l'equivalence pour reecrire
theorem use_iff (hpq : p <-> q) (hq : q) : p :=
  hpq.mpr hq

## 7. Egalite et Substitution

### 7.1 Le type `Eq` (egalite)

L'egalite `a = b` est une proposition fondamentale. Elle est reflexive (`rfl`), symetrique et transitive.

In [ ]:
#check Eq            -- Eq : {a : Sort u} -> a -> a -> Prop
#check (1 = 1)       -- 1 = 1 : Prop
#check (1 = 2)       -- 1 = 2 : Prop (mais non prouvable!)

-- Reflexivite : rfl
#check @rfl          -- rfl : a = a

-- Exemples
theorem one_eq_one : 1 = 1 := rfl
theorem two_plus_two : 2 + 2 = 4 := rfl   -- Lean calcule!

-- Symetrie et transitivite
#check @Eq.symm      -- Eq.symm : a = b -> b = a
#check @Eq.trans     -- Eq.trans : a = b -> b = c -> a = c

### 7.2 Substitution

Le principe de substitution : si `a = b`, alors on peut remplacer `a` par `b` dans n'importe quel contexte.

In [ ]:
-- Substitution avec le triangle
-- h \t e remplace a par b dans e en utilisant h : a = b
variable (a b : Nat) (h : a = b)
variable (P : Nat -> Prop) (hp : P a)

-- De P a et a = b, deduire P b
theorem subst_example : P b := h ▸ hp

-- Congruence : si a = b alors f a = f b
#check @congrArg     -- congrArg : (f : a -> b) -> x = y -> f x = f y

theorem congr_example (f : Nat -> Nat) (h : a = b) : f a = f b :=
  congrArg f h

-- Exemple concret
theorem add_congr (a b c : Nat) (h : a = b) : a + c = b + c :=
  congrArg (· + c) h

## 8. Preuves Calculatoires avec `calc`

### 8.1 La construction `calc`

Pour les chaines d'egalites ou d'inegalites, `calc` permet d'ecrire des preuves lisibles etape par etape.

In [ ]:
-- Exemple simple avec calc
theorem calc_example (a b c : Nat) (h1 : a = b) (h2 : b = c) : a = c :=
  calc a = b := h1
       _ = c := h2

-- Calcul arithmetique
theorem arith_calc : (2 + 3) * 4 = 20 :=
  calc (2 + 3) * 4
       = 5 * 4 := rfl
     _ = 20    := rfl

-- Preuve algebrique
theorem double_sum (n : Nat) : n + n = 2 * n :=
  calc n + n
       = 1 * n + 1 * n := by simp [Nat.one_mul]
     _ = (1 + 1) * n   := by ring
     _ = 2 * n         := rfl

### 8.2 calc avec plusieurs relations

In [ ]:
-- calc peut melanger = et <= etc.
theorem mixed_calc (a b c : Nat) (h1 : a = b) (h2 : b <= c) : a <= c :=
  calc a = b  := h1
       _ <= c := h2

-- Preuve plus complexe
theorem calc_complex (a b c d : Nat)
  (h1 : a = b) (h2 : c = d) (h3 : b + c = 10) : a + d = 10 :=
  calc a + d
       = b + d := congrArg (· + d) h1
     _ = b + c := congrArg (b + ·) h2.symm
     _ = 10    := h3

## 9. Logique Classique vs Constructive

### 9.1 La logique constructive par defaut

Par defaut, Lean utilise une **logique constructive** (ou intuitionniste). Cela signifie que certaines "lois" de la logique classique ne sont pas admises sans justification :

- **Tiers exclu** : `P \/ ¬P` n'est pas prouvable en general
- **Double negation** : `¬¬P -> P` n'est pas prouvable en general
- **Preuve par contradiction** : pour prouver `P`, on ne peut pas juste supposer `¬P` et deriver `False`

**Pourquoi ?** En logique constructive, prouver `P \/ Q` necessite de savoir **lequel** est vrai. Pour les propositions non decidables, ce n'est pas toujours possible.

In [ ]:
variable (p : Prop)

-- En logique constructive, ces theoremes NE SONT PAS prouvables sans hypothese :
-- theorem em_attempt : p \/ ¬p := sorry  -- Impossible sans Classical
-- theorem dne_attempt : ¬¬p -> p := sorry  -- Impossible sans Classical

-- Ce qui EST prouvable constructivement :

-- Introduction de la double negation
theorem dne_intro : p -> ¬¬p :=
  fun hp hnp => hnp hp

-- Triple negation = simple negation
theorem triple_neg : ¬¬¬p -> ¬p :=
  fun h hp => h (fun hnp => hnp hp)

### 9.2 Activer la logique classique

Pour utiliser le tiers exclu et les autres principes classiques, on importe `Classical` ou on utilise `open Classical`.

In [ ]:
open Classical

variable (p q : Prop)

-- Le tiers exclu est maintenant disponible
#check em p          -- em p : p \/ ¬p

-- Elimination de la double negation
theorem dne : ¬¬p -> p :=
  fun hnnp =>
    Or.elim (em p)
      (fun hp => hp)           -- Cas p vrai
      (fun hnp => absurd hnp hnnp)  -- Cas ¬p (contradiction)

-- Preuve par contradiction
theorem by_contradiction (h : ¬p -> False) : p :=
  dne (fun hnp => h hnp)

### 9.3 Lois de De Morgan

In [ ]:
open Classical

variable (p q : Prop)

-- De Morgan 1 : ¬(p \/ q) <-> ¬p /\ ¬q (constructif)
theorem de_morgan_1 : ¬(p \/ q) <-> ¬p /\ ¬q :=
  ⟨fun h => ⟨fun hp => h (Or.inl hp), fun hq => h (Or.inr hq)⟩,
   fun h hpq => hpq.elim h.left h.right⟩

-- De Morgan 2 : ¬(p /\ q) <-> ¬p \/ ¬q (necessite Classical!)
theorem de_morgan_2 : ¬(p /\ q) <-> ¬p \/ ¬q :=
  ⟨fun h => Or.elim (em p)
    (fun hp => Or.inr (fun hq => h ⟨hp, hq⟩))
    (fun hnp => Or.inl hnp),
   fun h hpq => h.elim (fun hnp => hnp hpq.left) (fun hnq => hnq hpq.right)⟩

## 10. Exercices

### Exercice 1 : Prouver l'associativite de Or

In [ ]:
variable (p q r : Prop)

-- Votre preuve ici
theorem or_assoc : (p \/ q) \/ r <-> p \/ (q \/ r) := sorry

### Exercice 2 : Distributivite de And sur Or

In [ ]:
variable (p q r : Prop)

-- p /\ (q \/ r) <-> (p /\ q) \/ (p /\ r)
theorem and_or_distrib : p /\ (q \/ r) <-> (p /\ q) \/ (p /\ r) := sorry

### Exercice 3 : Contraposee (classique)

In [ ]:
open Classical
variable (p q : Prop)

-- (p -> q) <-> (¬q -> ¬p)
theorem contrapositive : (p -> q) <-> (¬q -> ¬p) := sorry

## Solutions des exercices

In [ ]:
variable (p q r : Prop)

-- Solution exercice 1
theorem or_assoc_sol : (p \/ q) \/ r <-> p \/ (q \/ r) :=
  ⟨fun h => h.elim
    (fun hpq => hpq.elim (Or.inl) (fun hq => Or.inr (Or.inl hq)))
    (fun hr => Or.inr (Or.inr hr)),
   fun h => h.elim
    (fun hp => Or.inl (Or.inl hp))
    (fun hqr => hqr.elim (fun hq => Or.inl (Or.inr hq)) Or.inr)⟩

-- Solution exercice 2
theorem and_or_distrib_sol : p /\ (q \/ r) <-> (p /\ q) \/ (p /\ r) :=
  ⟨fun h => h.right.elim
    (fun hq => Or.inl ⟨h.left, hq⟩)
    (fun hr => Or.inr ⟨h.left, hr⟩),
   fun h => h.elim
    (fun hpq => ⟨hpq.left, Or.inl hpq.right⟩)
    (fun hpr => ⟨hpr.left, Or.inr hpr.right⟩)⟩

-- Solution exercice 3
open Classical in
theorem contrapositive_sol : (p -> q) <-> (¬q -> ¬p) :=
  ⟨fun h hnq hp => hnq (h hp),
   fun h hp => Or.elim (em q) id (fun hnq => absurd hp (h hnq))⟩

## Resume

| Concept | Description | Construction |
|---------|-------------|-------------|
| **Prop** | Univers des propositions | `Prop : Type` |
| **Implication** | `P -> Q` | `fun hp => ...` |
| **Conjonction** | `P /\ Q` | `⟨hp, hq⟩` ou `And.intro` |
| **Disjonction** | `P \/ Q` | `Or.inl hp` ou `Or.inr hq` |
| **Negation** | `¬P` = `P -> False` | `fun hp => contradiction` |
| **Equivalence** | `P <-> Q` | `⟨fun hp => ..., fun hq => ...⟩` |
| **Egalite** | `a = b` | `rfl` (reflexivite) |
| **calc** | Chaines de preuves | `calc a = b := ... _ = c := ...` |
| **Classique** | Tiers exclu | `open Classical` + `em p` |

### Prochaine etape

Dans le notebook **Lean-4-Quantifiers**, nous etendrons ces concepts aux **quantificateurs** (pour tout, il existe), permettant de raisonner sur des ensembles infinis d'objets.

---

*Notebook base sur "TP - Z3 - Tweety - Lean.pdf" Section VI.B.2 et adapte pour Lean 4*